In [2]:
import pandas as pd
import numpy as np
import os
import glob
import json
import pickle_rid_grabber as pg
import logging
import os

# logging.basicConfig(filename='tmp.log',
#                     format='%(levelname)s %(asctime)s :: %(message)s',
#                     level=logging.DEBUG)

with open('./config/config.json') as f:
    config = json.load(f)

def get_dtslug(doctype: str):
    doctype = doctype.replace('/', '')
    return doctype.replace(' ', '_').lower()

def remove_rids_for_test(filename,pkl_list,test_list):
    if len(pkl_list) > 0:

        elements = []
        for rid in pkl_list:
            if rid not in test_list:
                elements.append(rid)
        #np.savetxt(filename, elements, fmt='%s')
        #logging.debug(f"DOCUMENT:{filename},COLLECTED:{len(pkl_list)}\
        #,TEST_SET:{len(test_list)},REMOVED:{len(pkl_list)-len(elements)}")
        return [{"DOC": filename, "TOTAL_RIDS_COLLECTED": len(pkl_list), "AFTER DELETION OF TEST_SET": len(elements),"TEST_SET": len(test_list)}]
    else:
        return [{"DOC": filename, "TOTAL_RIDS_COLLECTED": len(pkl_list), "AFTER DELETION OF TEST_SET": len(elements),"TEST_SET": len(test_list)}]

        

def uniquing(doctypes: list):
    return list(set(doctypes))

all_docs=[
                    "UNDERWRITING TRANSMITTAL - 1008", 
               "CLOSING DISCLOSURE",
               "DEMOGRAPHIC ADDENDUM",
               "1003" 
       ]

# try:
#     for docname in all_docs:
#         rid_list = pg.download_list(docname)
#         pkl_filename = f"./rids_to_process/{str.upper(get_dtslug(docname))}"
#         test_filename = f"./rids_to_process/test_sets/{str.upper(get_dtslug(docname))}"
#         temp = pd.read_csv(test_filename,names=["rid"])
#         test_list = temp.rid.unique()
#         #remove_rids_for_test(pkl_filename,rid_list,test_list)
#         #np.savetxt(filename, rid_list, fmt='%s')
#         catch = remove_rids_for_test(pkl_filename,rid_list,test_list)
#         temp = pd.DataFrame(catch)
#         stats_list = pd.concat([stats_list,temp],axis=0)
#     print("Failed to process the doc: ",docname)
stats_list = pd.DataFrame()
for docname in all_docs:
    rid_list = pg.download_list(docname)
    pkl_filename = f"./rids_to_process/{str.upper(get_dtslug(docname))}"
    test_filename = f"./rids_to_process/test_sets/{str.upper(get_dtslug(docname))}"
    temp = pd.read_csv(test_filename,names=["rid"])
    test_list = temp.rid.unique()
    #remove_rids_for_test(pkl_filename,rid_list,test_list)
    #np.savetxt(filename, rid_list, fmt='%s')
    catch = remove_rids_for_test(pkl_filename,rid_list,test_list)
    temp = pd.DataFrame(catch)
    stats_list = pd.concat([stats_list,temp],axis=0)





23398
53856
26736
48300


In [3]:
stats_list

,DOC,TOTAL_RIDS_COLLECTED,AFTER DELETION OF TEST_SET,TEST_SET
0,./rids_to_process/UNDERWRITING_TRANSMITTAL_-_1008,23398,23251,147
0,./rids_to_process/CLOSING_DISCLOSURE,53856,53615,241
0,./rids_to_process/DEMOGRAPHIC_ADDENDUM,26736,26586,150
0,./rids_to_process/1003,48300,48053,247


In [6]:
stats_list[stats_list["AFTER DELETION OF TEST_SET"]>1000]

,DOC,TOTAL_RIDS_COLLECTED,AFTER DELETION OF TEST_SET,TEST_SET
0,./rids_to_process/BANK_STATEMENT,2651,2651,486
0,./rids_to_process/PAYOFF_STATEMENT,1011,1011,317
0,./rids_to_process/UNDISCLOSED_DEBT_MONITORING,2984,2849,135
0,./rids_to_process/MORTGAGE_STATEMENT,1542,1496,94
0,./rids_to_process/TITLE_COMMITMENT,1606,1526,117
0,./rids_to_process/DRIVERS_LICENSE,1614,1539,96
0,./rids_to_process/TAX_CERTIFICATE,1909,1909,28


In [7]:
stats_list.to_csv("caliber-train.csv",index=False)